In [ ]:
! pip install -q -U bitsandbytes
! pip install -q -U git+https://github.com/huggingface/transformers.git
! pip install -q -U git+https://github.com/huggingface/peft.git
#pip install -q -U git+https://github.com/huggingface/accelerate.git
#current version of Accelerate on GitHub breaks QLoRa
#Using standard pip instead
! pip install -q -U accelerate
! pip install -q -U datasets

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
! pip install trl

## dataframe

In [ ]:
import pandas as pd
df = pd.read_csv("rm_train.csv")
df

,Sr no.,Prompt,Questions,Option A,Option B,Option C,Option D,Correct Ans,small_explanation,Rejected_explanation
0,1,Select the most appropriate response from the ...,A market where there is no restrictions on the...,Regulated market,Unregulated market,Spot market,Future market,(b) Regulated market,An unregulated market is a market where transa...,Correct Ans:\n(c) Spot market\nExplanation for...
1,2,Carefully analyze the given options and select...,"Under perfect competition, price elasticity of...",Large,Slight,Infinite,Extreme,(c) Infinite,"Under perfect competition, a firm faces a hori...",correct Ans:\n(c) Infinite\nExplanation for th...
2,3,Consider the provided choices and identify the...,During recession the employment rate ---------...,Rises-falls,Rises-rises,falls-rises,Falls-falls,(d) Falls-falls,"During a recession, both the employment rate a...",Correct Ans:\n(c) falls-rises\n### Explanation...
3,4,Examine the given options thoroughly and deter...,The internal cause of business cycle is-,Technology shocks,Fluctuation in effective demand,Post war reconstruction,Population Growth,(b) Fluctuation in effective demand,Correct Answer: (b) Fluctuation in effective d...,Correct Ans:\n(c) Post war reconstruction\n###...
4,5,Choose the best option from the following stat...,An unemployment type cause due to structural c...,Involuntary,Ethical friction,Full employment,Structural,(d) Structural,Correct Answer: (d) Structural\n\nExplanation:...,Correct Ans:\n(b) Ethical friction\n### Explan...
5,6,Select the most appropriate response from the ...,At ‘trough’ the production in the economy reac...,High,Low,Constant,Negative,(b) Low,"Correct Answer: (b) Low\n\nExplanation: The ""t...",Correct Ans:\n(d) Negative\n### Explanation fo...
6,7,Carefully analyze the given options and select...,According to some economists which is not the ...,Fluctuations in investment,Micro Economic policy,Impact on aggregate economic activity,Psychological factors,(b) Micro Economic policy,Correct Answer: (b) Micro Economic policy\n\nE...,Correct Ans:\n(c) Impact on aggregate economic...
7,8,Consider the provided choices and identify the...,The four phases of Business Cycle are-,"Peak, Construction , Depression and Boom","Prosperity, Recession, Depression and Expansion","Boom, Downswing , Expansion and Prosperity","Peak, Recession , Trough and Recovery","(d) Peak, Recession , Trough and Recovery","Correct Answer: (d) Peak, Recession, Trough, a...","(a) Peak, Construction , Depression and Boom\n..."
8,9,Examine the given options thoroughly and deter...,Chinas recent slowdown causes-,Cycle of decline and panic across the world,Countries across the Globe were able to insula...,Stock markets in the Emerging Economics largel...,Old technology fuelled the economic decline,(a) Cycle of decline and panic across the world,Correct Answer: (a) Cycle of decline and panic...,Correct Ans:\n(d) Old technology fuelled the e...
9,10,Choose the best option from the following stat...,Friction unemployment is the characteristic of...,Expansion,Peak,Construction,Recovery,(a) Expansion,Correct Answer: (a) Expansion\n\nExplanation: ...,Correct Ans:\n(a) Expansion\n### Explanation f...


## Use Pre-trained Reward Model

In [ ]:
# from transformers import AutoModelForSequenceClassification, AutoTokenizer
# reward_name = "OpenAssistant/reward-model-deberta-v3-large"
# rank_model, tokenizer1 = AutoModelForSequenceClassification.from_pretrained(reward_name), AutoTokenizer.from_pretrained(reward_name)

In [ ]:
l = []
def reward_model() :
    for i in range(0,50) :
        question = " Instruction : " + str(df.iloc[i,1]) + "question :" + str(df.iloc[i,2]) + "option A : " +  str(df.iloc[i,3]) + "option B :" + str(df.iloc[i,4]) + "option c : " + str(df.iloc[i,5]) + "option D : " +  str(df.iloc[i,6])
        answer = df.iloc[i,9]
        inputs = tokenizer(question, answer, return_tensors='pt')
        score = model(**inputs).logits[0].cpu().detach()
        l.append(score)

reward_model()
df['reward_finetuned'] = l

In [ ]:
from huggingface_hub import notebook_login
notebook_login()

In [ ]:
from peft import LoraConfig
from transformers import AutoModelForCausalLM
from peft import PeftModel
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from accelerate import dispatch_model, infer_auto_device_map

base_model_name_or_path = "bigscience/bloom-3b"
HUGGING_FACE_USER_NAME = "aayushi135"
model_name = "bloom3b_finetuned_1000"
repo_name = f"{HUGGING_FACE_USER_NAME}/{model_name}"

config = LoraConfig.from_pretrained(repo_name)

ppo_model = AutoModelForCausalLM.from_pretrained(
    config.base_model_name_or_path,
    device_map="auto",
    torch_dtype=torch.bfloat16,
    trust_remote_code=True,
)
# Load the LoRA model
inference_model = PeftModel.from_pretrained(ppo_model, repo_name)   # <-- e
tokenizer = AutoTokenizer.from_pretrained(config.base_model_name_or_path)

adapter_config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/693 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/6.01G [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/19.7M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/222 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/85.0 [00:00<?, ?B/s]

In [ ]:
# device = torch.device("cuda")
# ppo_model.cuda()
# torch.cuda.set_device(0)
device = torch.device('cuda:0')  # Choose the target device, e.g., cuda:0
ppo_model.to(device)

BloomForCausalLM(
  (transformer): BloomModel(
    (word_embeddings): Embedding(250880, 2560)
    (word_embeddings_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
    (h): ModuleList(
      (0-29): 30 x BloomBlock(
        (input_layernorm): LayerNorm((2560,), eps=1e-05, elementwise_affine=True)
        (self_attention): BloomAttention(
          (query_key_value): Linear(
            in_features=2560, out_features=7680, bias=True
            (lora_dropout): ModuleDict(
              (default): Dropout(p=0.05, inplace=False)
            )
            (lora_A): ModuleDict(
              (default): Linear(in_features=2560, out_features=16, bias=False)
            )
            (lora_B): ModuleDict(
              (default): Linear(in_features=16, out_features=7680, bias=False)
            )
            (lora_embedding_A): ParameterDict()
            (lora_embedding_B): ParameterDict()
          )
          (dense): Linear(in_features=2560, out_features=2560, bias=True)


## PPO Training

In [ ]:
import trl
from trl import AutoModelForCausalLMWithValueHead, PPOConfig, PPOTrainer

ppo_model = AutoModelForCausalLMWithValueHead.from_pretrained(ppo_model,torch_dtype=torch.bfloat16,is_trainable=True)
ref_model = trl.create_reference_model(ppo_model)

In [ ]:
ppo_config = PPOConfig(
    batch_size=1,
)

In [ ]:
ppo_trainer = PPOTrainer(ppo_config, ppo_model, ref_model, tokenizer)

In [ ]:
from trl.core import respond_to_batch

In [ ]:
import torch
import os
# if args.gpu and torch.cuda.is_available():
#         model.cuda()
#         criterion = criterion.cuda()
for i in range(0,50) :
  # with torch.cuda.amp.autocast():
    query = " Instruction : " + str(df.iloc[i,1]) + "question :" + str(df.iloc[i,2]) + "option A : " +  str(df.iloc[i,3]) + "option B :" + str(df.iloc[i,4]) + "option c : " + str(df.iloc[i,5]) + "option D : " +  str(df.iloc[i,6]) + str(df.iloc[i,9])
    query_tensor = tokenizer.encode(query, return_tensors="pt").to(device)
    response_tensor  = respond_to_batch(ppo_model, query_tensor)
    reward = [df.iloc[i,10]]
    ppo_trainer.step([query_tensor[0]], [response_tensor[0]], reward)

## now again calculating the reward after RLHF

In [ ]:
df

,Sr no.,Prompt,Questions,Option A,Option B,Option C,Option D,Correct Ans,small_explanation,Rejected_explanation,reward_finetuned
0,1,Select the most appropriate response from the ...,A market where there is no restrictions on the...,Regulated market,Unregulated market,Spot market,Future market,(b) Regulated market,An unregulated market is a market where transa...,Correct Ans:\n(c) Spot market\nExplanation for...,[tensor(3.4035)]
1,2,Carefully analyze the given options and select...,"Under perfect competition, price elasticity of...",Large,Slight,Infinite,Extreme,(c) Infinite,"Under perfect competition, a firm faces a hori...",correct Ans:\n(c) Infinite\nExplanation for th...,[tensor(2.4808)]
2,3,Consider the provided choices and identify the...,During recession the employment rate ---------...,Rises-falls,Rises-rises,falls-rises,Falls-falls,(d) Falls-falls,"During a recession, both the employment rate a...",Correct Ans:\n(c) falls-rises\n### Explanation...,[tensor(0.4574)]
3,4,Examine the given options thoroughly and deter...,The internal cause of business cycle is-,Technology shocks,Fluctuation in effective demand,Post war reconstruction,Population Growth,(b) Fluctuation in effective demand,Correct Answer: (b) Fluctuation in effective d...,Correct Ans:\n(c) Post war reconstruction\n###...,[tensor(3.3573)]
4,5,Choose the best option from the following stat...,An unemployment type cause due to structural c...,Involuntary,Ethical friction,Full employment,Structural,(d) Structural,Correct Answer: (d) Structural\n\nExplanation:...,Correct Ans:\n(b) Ethical friction\n### Explan...,[tensor(-1.8425)]
5,6,Select the most appropriate response from the ...,At ‘trough’ the production in the economy reac...,High,Low,Constant,Negative,(b) Low,"Correct Answer: (b) Low\n\nExplanation: The ""t...",Correct Ans:\n(d) Negative\n### Explanation fo...,[tensor(4.1192)]
6,7,Carefully analyze the given options and select...,According to some economists which is not the ...,Fluctuations in investment,Micro Economic policy,Impact on aggregate economic activity,Psychological factors,(b) Micro Economic policy,Correct Answer: (b) Micro Economic policy\n\nE...,Correct Ans:\n(c) Impact on aggregate economic...,[tensor(-0.8979)]
7,8,Consider the provided choices and identify the...,The four phases of Business Cycle are-,"Peak, Construction , Depression and Boom","Prosperity, Recession, Depression and Expansion","Boom, Downswing , Expansion and Prosperity","Peak, Recession , Trough and Recovery","(d) Peak, Recession , Trough and Recovery","Correct Answer: (d) Peak, Recession, Trough, a...","(a) Peak, Construction , Depression and Boom\n...",[tensor(-3.8721)]
8,9,Examine the given options thoroughly and deter...,Chinas recent slowdown causes-,Cycle of decline and panic across the world,Countries across the Globe were able to insula...,Stock markets in the Emerging Economics largel...,Old technology fuelled the economic decline,(a) Cycle of decline and panic across the world,Correct Answer: (a) Cycle of decline and panic...,Correct Ans:\n(d) Old technology fuelled the e...,[tensor(-0.8192)]
9,10,Choose the best option from the following stat...,Friction unemployment is the characteristic of...,Expansion,Peak,Construction,Recovery,(a) Expansion,Correct Answer: (a) Expansion\n\nExplanation: ...,Correct Ans:\n(a) Expansion\n### Explanation f...,[tensor(1.3280)]


In [ ]:
from datasets import load_dataset
data = load_dataset("csv", data_files="rm_train.csv")
# data = data.to(device)
data #= data.map(lambda samples: tokenizer(samples["quote"]), batched=True)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['Sr no.', 'Prompt', 'Questions', 'Option A', 'Option B', 'Option C', 'Option D', 'Correct Ans', 'small_explanation', 'Rejected_explanation'],
        num_rows: 50
    })
})

In [ ]:
def generate_prompt(data_point):
    return f""" Examine the given options thoroughly and determine the optimal choice that accurately addresses the following multiple-choice question.
    ### Question:
    {data_point['Questions']}
    ### Option A:
    {data_point['Option A']}
    ### Option B:
    {data_point['Option B']}
    ### Option C:
    {data_point['Option C']}
    ### Option D:
    {data_point['Option D']}"""

mapped_qa_dataset = data.map(lambda samples: tokenizer(generate_prompt(samples)))

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

In [ ]:
query = " Instruction : " + str(df.iloc[1,1]) + "question :" + str(df.iloc[1,2]) + "option A : " +  str(df.iloc[1,3]) + "option B :" + str(df.iloc[1,4]) + "option c : " + str(df.iloc[1,5]) + "option D : " +  str(df.iloc[1,6])
query

' Instruction : Carefully analyze the given options and select the best answer that accurately addresses the following multiple-choice question.question :Under perfect competition, price elasticity of demand of a firm is-option A : Largeoption B :Slightoption c : Infiniteoption D : Extreme'

In [ ]:
query_tensor = tokenizer.encode(query, return_tensors="pt").to(device)
output_tokens = ppo_model.generate(query_tensor, max_new_tokens=200).to(device)

In [ ]:
(tokenizer.decode(output_tokens[0], skip_special_tokens=True))

' Instruction : Carefully analyze the given options and select the best answer that accurately addresses the following multiple-choice question.question :Under perfect competition, price elasticity of demand of a firm is-option A : Largeoption B :Slightoption c : Infiniteoption D : Extremeallicense u and v to a firm each in a town. Firm a supplies its output at pA and firm b supplies its output at PA. If the average price of output in the market is PA, then the price elasticity of demand of firm a is\nOption A: Large Option B: Slight Option C: Infinite Option D: Extraneous Let xA and xB be the quantities of output offered by firm a and firm b, respectively, under perfect competition. The price elasticity of demand of firm a is\nOption A: Large Option B: Slight Option C: Infinite Option D: Extraneous Let xA and xB be the quantities of output offered by firm a and firm b, respectively, under perfect competition. The price elasticity of demand of firm b is\nOption A: Large Option B: Sligh

In [ ]:
tokenizer1 = AutoTokenizer.from_pretrained("facebook/opt-350m")
question = " Instruction : " + str(df.iloc[1,1]) + "question :" + str(df.iloc[1,2]) + "option A : " +  str(df.iloc[1,3]) + "option B :" + str(df.iloc[1,4]) + "option c : " + str(df.iloc[1,5]) + "option D : " +  str(df.iloc[1,6])
# answer = "Correct Ans: (c) Spot market\n    Explanation for the correct ans : A market where there are no restrictions on the transactions is called a spot market. It is not regulated or specified. Spot markets are usually regulated by the government. They specify the limits and prices for trading. Spot markets are usually daily or weekly. Spot markets are usually regulated by the government. They specify the limits and prices for trading. Explanation for the incorrect ans : A market where there are no restrictions on the transactions is called a regulated market or specified market. It is not called a spot market. It is usually regulated by the government. It specifies the limits and prices for trading. It is usually daily or weekly. It is usually regulated by the government. It specifies the limits and prices for trading. It is usually daily or weekly. It is usually regulated by the government. It specifies the limits and prices for trading. It is usually daily or weekly. It is usually regulated by the government."
answer = "Firm a supplies its output at pA and firm b supplies its output at PA. If the average price of output in the market is PA, then the price elasticity of demand of firm a is\nOption A: Large Option B: Slight Option C: Infinite Option D: Extraneous Let xA and xB be the quantities of output offered by firm a and firm b, respectively, under perfect competition. The price elasticity of demand of firm a is\nOption A: Large Option B: Slight Option C: Infinite Option D: Extraneous Let xA and xB be the quantities of output offered by firm a and firm b, respectively, under perfect competition. The price elasticity of demand of firm b is\nOption A: Large Option B: Slight Option C: Infinite Option D: Extraneous Let xA and xB be the quantities of output offered by firm a and firm b, respectively, under perfect competition. The"
inputs = tokenizer1(question, answer, return_tensors='pt')
score = model(**inputs).logits[0].cpu().detach()
score

tensor([0.5713])

In [ ]:
tokenizer1 = AutoTokenizer.from_pretrained("facebook/opt-350m")
question = " Instruction : " + str(df.iloc[1,1]) + "question :" + str(df.iloc[1,2]) + "option A : " +  str(df.iloc[1,3]) + "option B :" + str(df.iloc[1,4]) + "option c : " + str(df.iloc[1,5]) + "option D : " +  str(df.iloc[1,6])
answer = "Correct Ans: (c) Spot market\n    Explanation for the correct ans : A market where there are no restrictions on the transactions is called a spot market. It is not regulated or specified. Spot markets are usually regulated by the government. They specify the limits and prices for trading. Spot markets are usually daily or weekly. Spot markets are usually regulated by the government. They specify the limits and prices for trading. Explanation for the incorrect ans : A market where there are no restrictions on the transactions is called a regulated market or specified market. It is not called a spot market. It is usually regulated by the government. It specifies the limits and prices for trading. It is usually daily or weekly. It is usually regulated by the government. It specifies the limits and prices for trading. It is usually daily or weekly. It is usually regulated by the government. It specifies the limits and prices for trading. It is usually daily or weekly. It is usually regulated by the government."
# answer = " Instruction : Carefully analyze the given options and select the best answer that accurately addresses the following multiple-choice question.question :Under perfect competition, price elasticity of demand of a firm is-option A : Largeoption B :Slightoption c : Infiniteoption D : Extremeallicense u and v to a firm each in a town. Firm a supplies its output at pA and firm b supplies its output at PA. If the average price of output in the market is PA, then the price elasticity of demand of firm a is\nOption A: Large Option B: Slight Option C: Infinite Option D: Extraneous Let xA and xB be the quantities of output offered by firm a and firm b, respectively, under perfect competition. The price elasticity of demand of firm a is\nOption A: Large Option B: Slight Option C: Infinite Option D: Extraneous Let xA and xB be the quantities of output offered by firm a and firm b, respectively, under perfect competition. The price elasticity of demand of firm b is\nOption A: Large Option B: Slight Option C: Infinite Option D: Extraneous Let xA and xB be the quantities of output offered by firm a and firm b, respectively, under perfect competition. The"
inputs = tokenizer1(question, answer, return_tensors='pt')
score = model(**inputs).logits[0].cpu().detach()
score

tensor([1.3762])